In [ ]:
import numpy as np

from syft.lib.adp.scalar import PhiScalar
from syft.lib.adp.entity import Entity
from syft.lib.adp.tensor import Tensor
from syft.lib.adp.adversarial_accountant import AdversarialAccountant
from syft.lib.adp.publish import publish

# Data Owner Loads in Private Data

In [ ]:
x = PhiScalar(0,0.01,1) #.tag('x').send(duet)
y = PhiScalar(0,0.02,1) #.tag('y').send(duet)
z = PhiScalar(0,0.02,1) #.tag('z').send(duet)

In [ ]:
entities = [Entity(unique_name="Tudor"), Entity(unique_name="Madhava"), Entity(unique_name="Kritika"), Entity(unique_name="George")]

a = Tensor(np.array([[1,1],[1,0],[0,1],[0,0]])).private(min_val=0, max_val=1, entities=entities, is_discrete=True)
# a = a.tag('a').send(duet)

b = Tensor(np.array([[1],[1],[0],[0]])).private(min_val=0, max_val=1, entities=entities, is_discrete=False)
# b = b.tag('b').send(duet)

# Data Scientist Connects with Domain (with new key)

Behind the scenes, when a data scientist queries a private tensor, if that verification key doesn't yet have an accountant - we need to initailize one for that person.

In [ ]:
# duet = sy.duet()
# x = duet.store['x']
# y = duet.store['y']
# z = duet.store['z']

In [ ]:
# p = x * x

# o = p + y*y + z
# z = o * o * o

o = x*x + y*y + z
z = o * o * o

In [ ]:
# z.get() # triggers an error 

In [ ]:
# z_public = z.publish(sigma=0.1)
# # BEHIND THE SCENES... it should try to find domain.accountants[verify_key] to 
# # pass into the actual private method

In [ ]:
# z_public.get() # will download automatically (no matter what)

# Local Demo Code

In [ ]:
acc = AdversarialAccountant(max_budget=10)
z.publish(acc=acc, sigma=0.2)

In [ ]:
# the data scientist should never see this
acc = AdversarialAccountant(max_budget=10)

publish([z,z], acc=acc, sigma=0.2)

In [ ]:
acc = AdversarialAccountant(max_budget=3000000)


entities = [Entity(unique_name="Tudor"), Entity(unique_name="Madhava"), Entity(unique_name="Kritika"), Entity(unique_name="George")]

x = Tensor(np.array([[1,1],[1,0],[0,1],[0,0]])).private(min_val=0, max_val=1, entities=entities, is_discrete=True)
y = Tensor(np.array([[1],[1],[0],[0]])).private(min_val=0, max_val=1, entities=entities, is_discrete=False)

_weights = Tensor(np.random.uniform(size=(2,1)))

In [ ]:
weights = _weights + 0
acc = AdversarialAccountant(max_budget=7)  # causes error at end of budget
# acc = AdversarialAccountant(max_budget=3000000)  

for i in range(10):
    batch_loss = 0

    pred = x.dot(weights)
    loss = np.mean(np.square(y-pred))
    loss.backward()

    weight_grad = (weights.grad * 0.5)
    weight_grad = weight_grad.publish(acc=acc, sigma=0.1)

    weights = weights - weight_grad
    batch_loss += loss.value

    acc.print_ledger()
#     print(weights)